#Welcome! 

This is an interactive Python notebook that has been made by Washieu Anan. For my project, I wanted to inform others on ways to improve their recycling habits. In this notebook, you will see my statistical analysis of the dataset that I retrieved from NYC OpenData. In this dataset, I was provided with the specific zones that garbage was collected in, the specific districts in which garbage was collected in, the fiscal month in which garbage was collected in, the fiscal year in which garbage was collected in, the total diversion rate of the garbage (per zone and district) and the capture rate of paper that was officially recycled (per zone and district). 

In order to understand the purpose of each section of code, please refer to the green lines of text above each section of code. The green lines will help you with understanding each section of my code. 


 

In [0]:
#Below in our first cell, we have imported many crucial Python libraries that are needed to best statistically analyze the data that we have.
import pandas as pd
import numpy as np
import random
import math
import itertools
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
pd.set_option('display.max_rows', None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#If you are using Google Colaboratory, please utilize this following cell to upload your .csv files onto your notebooks. 
from google.colab import files
uploaded = files.upload()

Saving Recycling_Diversion_and_Capture_Rates.csv to Recycling_Diversion_and_Capture_Rates.csv


In [0]:
#This following cell tells the computer to take the dataset into the computer and store it as variable. 
df = pd.read_csv("Recycling_Diversion_and_Capture_Rates.csv")

In [0]:
#We did not necessairly have to use the .head() function. We merely wanted to test whether or not our dataset was officially inputted into the system. 
df.head()

,Zone,District,Fiscal Month Number,Fiscal Year,Month Name,Diversion Rate-Total (Total Recycling / Total Waste),Capture Rate-Paper (Total Paper / Max Paper),Capture Rate-MGP (Total MGP / Max MGP),Capture Rate-Total ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100
0,Brooklyn North,BKN01,10,2019,April,14.7,44.9,43.0,44.1
1,Brooklyn North,BKN02,10,2019,April,20.0,34.2,57.9,41.2
2,Brooklyn North,BKN03,10,2019,April,12.2,33.5,44.9,38.2
3,Brooklyn North,BKN04,10,2019,April,15.5,35.2,68.5,48.8
4,Brooklyn North,BKN05,10,2019,April,10.1,22.3,45.1,31.5


In [0]:
#In this cell, we are listing all of the columns within the dataset, for which we have already done in our inductory segment. 
df.columns

Index(['Zone', 'District', 'Fiscal Month Number', 'Fiscal Year', 'Month Name',
       'Diversion Rate-Total (Total Recycling / Total Waste)',
       'Capture Rate-Paper (Total Paper / Max Paper)',
       'Capture Rate-MGP (Total MGP / Max MGP)',
       'Capture Rate-Total ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100'],
      dtype='object')

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2832 entries, 0 to 2831
Data columns (total 9 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   Zone                                                                                     2832 non-null   object 
 1   District                                                                                 2832 non-null   object 
 2   Fiscal Month Number                                                                      2832 non-null   int64  
 3   Fiscal Year                                                                              2832 non-null   int64  
 4   Month Name                                                                               2832 non-null   object 
 5   Diversion Rate-Total (Total Recycling / Total Waste)           

In [0]:
#In this cell, we are telling the computer to read each columns as a string of letters rather than a number that has been located within an array. 
df['Fiscal_Date'] = df['Fiscal Year'].astype(str) + '-' + df['Fiscal Month Number'].astype(str)
df['Fiscal_Date'] = pd.to_datetime(df['Fiscal_Date'])
df['Normal_Date'] = df['Fiscal Year'].astype(str) + '-' + df['Month Name']
df['Normal_Date'] = pd.to_datetime(df['Normal_Date'])


In [0]:
#In this cell, we outline the specific zones that are available. 
df.Zone.unique()

array(['Brooklyn North', 'Brooklyn South', 'Bronx', 'Manhattan',
       'Queens East', 'Queens West', 'Staten Island'], dtype=object)

In [0]:
#In this cell, we are renaming each of the zones to look much cleaner in appearance when being shown to the public. 
df_brooklyn_north = df[df['Zone'] == 'Brooklyn North']
df_brooklyn_south = df[df['Zone'] == 'Brooklyn South']
df_bronx = df[df['Zone'] == 'Bronx']
df_manhattan = df[df['Zone'] == 'Manhattan']
df_queens_east = df[df['Zone'] == 'Queens East']
df_queens_west = df[df['Zone'] == 'Queens West']
df_staten_island = df[df['Zone'] == 'Staten Island']

In [0]:
#In this cell, we are creating a list in which we list all of the specific zones. 
zone_list = [df_brooklyn_north, df_brooklyn_south, df_bronx, df_manhattan, df_queens_east, df_queens_west, df_staten_island]

In [0]:
#In this cell, we are telling the computer to go through each zone. While going through each zone, 
#we are telling the commputer to define the shape (the number of rows) and the name of the zone that corresponds with it. 
for df_zone in zone_list:
    shape = df_zone.shape
    zone_name = df_zone.Zone.unique()[0]
    print(zone_name, shape[0])
    

Brooklyn North 432
Brooklyn South 432
Bronx 576
Manhattan 576
Queens East 336
Queens West 336
Staten Island 144


In [0]:
#In this cell, we are telling the computer to tell us the number of districts within this dataset. 
districts = []
for zone in df.Zone.unique():
    zone_df = df[df['Zone'] == zone]
    count = 0
    for district in zone_df.District.unique():
      districts.append(district)
      
len(districts)

59

In [0]:
#In this cell, we are telling the computer to go through each zone and district and create a "scatter plot" that contains a line for the:
#Total Diversion Rate, Total Capture Rate of Paper, Total Capture Rate of Metal, Glass, and Plastic, Overall Capture Rate. 
#If you hover over a line, it will provide you with the specific name of the line that you are hovering over. 
for zone in df.Zone.unique():
    zone_df = df[df['Zone'] == zone]
    count = 0
    for district in zone_df.District.unique():
        district_df = zone_df[zone_df['District'] == district]
        district_df = district_df.sort_values(by=['Normal_Date'])
        fig = go.Figure()
        fig.add_trace(go.Scatter(
                        x=district_df.Normal_Date,
                        y=district_df['Diversion Rate-Total (Total Recycling / Total Waste)'],
                        name=f"'Diversion Rate-Total' - {district_df.District.unique()[0]}",
                        line_color='deepskyblue',
                        opacity=0.8))

        fig.add_trace(go.Scatter(
                        x=district_df.Normal_Date,
                        y=district_df['Capture Rate-Paper (Total Paper / Max Paper)'],
                        name=f"'Capture Rate-Paper' - {district_df.District.unique()[0]}",
                        line_color='firebrick',
                        opacity=0.8))

        fig.add_trace(go.Scatter(
                        x=district_df.Normal_Date,
                        y=district_df['Capture Rate-MGP (Total MGP / Max MGP)'],
                        name=f"'Capture Rate-MGP' - {district_df.District.unique()[0]}",
                        line_color='green',
                        opacity=0.8))
        
        fig.add_trace(go.Scatter(
                        x=district_df.Normal_Date,
                        y=district_df['Capture Rate-Total ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100'],
                        name=f"'Capture Rate-Total' - {district_df.District.unique()[0]}",
                        line_color='rgb(102,102,255)',
                        opacity=0.8))

        fig.update_layout(xaxis_range=[min(district_df.Normal_Date),max(district_df.Normal_Date)],
                          title_text=f"'Rates for District ' + {district_df.District.unique()[0]}",xaxis_title="Date",
            yaxis_title="Rate",
                        xaxis_rangeslider_visible=True)
        fig.show()
        


In [0]:
#In this cell, we are finding the mean, standard deviation, minimum, maximum, and percentages of the data to better analyze the type of data that we are looking at. 
for zone in df.Zone.unique():
    zone_df = df[df['Zone'] == zone]
    count = 0
    for district in zone_df.District.unique():
        district_df = zone_df[zone_df['District'] == district]
        district_df = district_df.sort_values(by=['Normal_Date'])
        print(district + '  statistics')
        display(district_df.describe())
        print('\n\n\n')

BKN01  statistics


,Fiscal Month Number,Fiscal Year,Diversion Rate-Total (Total Recycling / Total Waste),Capture Rate-Paper (Total Paper / Max Paper),Capture Rate-MGP (Total MGP / Max MGP),Capture Rate-Total ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,6.500000,2017.500000,15.354167,48.291667,45.754167,47.129167
std,3.488583,1.129865,0.917202,4.008174,2.807131,2.605229
min,1.000000,2016.000000,13.500000,40.500000,39.900000,41.300000
25%,3.750000,2016.750000,14.700000,45.350000,43.925000,45.450000
50%,6.500000,2017.500000,15.350000,48.250000,46.050000,47.400000
75%,9.250000,2018.250000,16.025000,51.125000,47.125000,48.850000
max,12.000000,2019.000000,17.100000,58.400000,55.400000,52.900000






BKN02  statistics


,Fiscal Month Number,Fiscal Year,Diversion Rate-Total (Total Recycling / Total Waste),Capture Rate-Paper (Total Paper / Max Paper),Capture Rate-MGP (Total MGP / Max MGP),Capture Rate-Total ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,6.500000,2017.500000,19.262500,33.537500,56.170833,40.016667
std,3.488583,1.129865,1.489056,2.025752,3.015701,2.041832
min,1.000000,2016.000000,16.500000,28.200000,49.100000,34.400000
25%,3.750000,2016.750000,17.900000,32.400000,54.125000,38.850000
50%,6.500000,2017.500000,19.450000,33.550000,56.450000,40.000000
75%,9.250000,2018.250000,20.425000,34.550000,57.925000,41.200000
max,12.000000,2019.000000,22.500000,38.000000,63.800000,43.900000






BKN03  statistics


...

,Fiscal Month Number,Fiscal Year,Diversion Rate-Total (Total Recycling / Total Waste),Capture Rate-Paper (Total Paper / Max Paper),Capture Rate-MGP (Total MGP / Max MGP),Capture Rate-Total ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,6.500000,2017.500000,20.156250,40.362500,75.895833,52.195833
std,3.488583,1.129865,1.508544,3.788483,6.201268,4.602218
min,1.000000,2016.000000,18.400000,34.600000,63.400000,44.400000
25%,3.750000,2016.750000,19.100000,37.875000,71.075000,49.200000
50%,6.500000,2017.500000,19.800000,39.400000,75.850000,51.100000
75%,9.250000,2018.250000,20.700000,42.950000,79.975000,54.375000
max,12.000000,2019.000000,27.100000,48.700000,90.400000,63.200000






SI02  statistics


,Fiscal Month Number,Fiscal Year,Diversion Rate-Total (Total Recycling / Total Waste),Capture Rate-Paper (Total Paper / Max Paper),Capture Rate-MGP (Total MGP / Max MGP),Capture Rate-Total ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,6.500000,2017.500000,19.983333,45.200000,75.625000,55.327083
std,3.488583,1.129865,1.815926,4.155591,5.734943,4.641521
min,1.000000,2016.000000,17.400000,37.700000,64.100000,47.200000
25%,3.750000,2016.750000,18.700000,42.275000,72.200000,52.575000
50%,6.500000,2017.500000,19.450000,44.000000,74.800000,54.050000
75%,9.250000,2018.250000,20.625000,48.350000,78.975000,57.950000
max,12.000000,2019.000000,27.300000,54.100000,88.100000,66.100000






SI03  statistics


,Fiscal Month Number,Fiscal Year,Diversion Rate-Total (Total Recycling / Total Waste),Capture Rate-Paper (Total Paper / Max Paper),Capture Rate-MGP (Total MGP / Max MGP),Capture Rate-Total ((Total Recycling - Leaves (Recycling)) / (Max Paper + Max MGP))x100
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,6.500000,2017.500000,21.095833,47.906250,81.429167,59.072917
std,3.488583,1.129865,2.031346,4.124778,6.349701,4.581403
min,1.000000,2016.000000,18.100000,40.500000,66.600000,51.200000
25%,3.750000,2016.750000,19.875000,44.375000,76.700000,56.375000
50%,6.500000,2017.500000,20.450000,46.900000,80.950000,58.000000
75%,9.250000,2018.250000,21.650000,51.875000,86.675000,62.300000
max,12.000000,2019.000000,29.500000,55.800000,95.400000,69.200000




# ***Analysis***

Using the graphs and charts that have been developed via our computer, we can now begin to analyze our data. 

A majority of the Browning community lives in the Upper East Side. The Upper East Side is considered District 08 in Manhattan. Below, the graph created for the Upper East Side will be shown: 
![alt text](https://drive.google.com/uc?id=155lPQT1FOWfSlGEj9z46c7nxRgz7CbYI)


Looking at this graph, we can understand that the Diversion Rate (for the Upper East Side) is considerably low. Less than 30% of all materials used in the Upper East Side is recycled. Such practices lead to an overall increase in pollution. By not recycling the materials that we use, we are exposing ourselves and other forms of life to harmful chemicals and particles that can pose a threat to our overall well-being. We are also overfilling our landfills and creating the destruction of our natural habitats. 

Many members of the Browning community also live in the Upper West Side. The Upper West Side is considered District 10 in Manhattan. Below, the graph created for the Upper West Wide will be shown:
![alt text](https://drive.google.com/uc?id=1ou-5v-3FIs4Hyce3OT-P_E3Eg3C97qMA)


Looking at this graph, we can see many similarties to the graph that was generated for the Upper East Side. The Diversion Rate Total is also considerably low with the Total Diversion Rate being less than 30%. 

Also, a common trend between both of the graphs is the relative time of spikes within the graph. Those spikes are often around the summer and early winter. Such spikes are understandable because of the common influx of tourists coming into New York City. Such tourists may not know of the certain ways in which they can deposit or manage the trash that they generate.

---
Using the information that we can see, here are some ways to make sure that our Total Diversion Rate increase. 


1.   Know what you can and cannot recycle:
        * New York has provided people with a complete list that outlines the specific rules and regulations on how to recycle a certain item ([Recycling Info](https://www1.nyc.gov/assets/dsny/site/services/recycling/what-to-recycle))
        * Many items have a symbol that implies whether an item is recyclable or not (♻️)

2.   Begin to purchase more recyceable items. 
        * The world around us is surrounded with technology. However, as more people begin to use technology, technological waste also begins to increase. 
            * If you have an old device that you do not use anymore, many companies have programs in which they will purchase your device from you and recycle it. Not only do you make extra money but you are also saving the world! Apple also utilizes many components of recycled phones (metal and glass) to utilize them when building new phones. 
            * Begin to purchase rechargeable batteries. Rechargeable batteries allow for an overall decrease of the amount of batteries thrown out within one household. In the long run, this helps save many people spend less when purchasing batteries because they will almost never have to purchase batteries again (until the rechargeable battery wears out). 
        * Begin to transition from one-time plastic items to reuseable fiber, metal, wooden items.
            * Many people are beginning to convert their usage of one-time plastic bottles to reuseable metal bottles. 
            * Many people are also beginning to transition from using one-time use plastic bag to resueable fiber bags. In particular, New York State has mandated a state-wide law on the banning on plastic bag. However, because this is a recent change (as of March 1, 2020), we have not been able to see the major effects that this law has put on the overall enviornment. 
            * As a community, a shift in the common items that we use can prove to be extremely helpful towards the world that we live in. 

3.   Begin to sort out your garbage. 
        * Create separate areas for regular trash and recycled trash. This will allow for both you and your family members to make concious decisions on what they are putting into the trash can. 
        * Also, in New York City, most streets have multiple slots for which you can deposit your trash based on the type of trash that you are throwing out. 
        * Also, if you have any clothes or items that you either are not interested in or do not wear anymore, please remember to donate such items. Not only do they help someone in need, but they also allow for the reduction of waste for such items. 


**Thank you for viewing my project! If you have any further questions, please email Washieu Anan (wanan@browning.edu).**












